### LOS Project (Limits of stability)
It is again data measured with force platform, but the procedure is a bit more dynamic. Subjects were asked to stay still for 10 seconds, and after that lean forward as far as possible without losing a balance and stay in this position for the remaining 20 sec. This procedure divides recording into 3 phases. Initial standing, leaning forward, and end position. I phase 1, and 3, paramaters describing changes in body's Center of pressure are measured. For each trial, a chart is being made and exported into charts folder for further inspection. Additionaly excel database is being exported. 

In [69]:
import pandas as pd
import numpy as np
import re
from scipy import signal
from sklearn.linear_model import LinearRegression
from pyentrp import entropy as ent
import matplotlib.pyplot as plt
import glob

In [70]:
# Class for text attributes (Important for aesthethics of print outs)
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [71]:
#! On Windows you need to replace all '/' with '\\'
patternFname = 'Data/LOS_Data/'
patternCharts = 'Charts/'
patternFiles = 'Data/LOS_Data/*.txt'
patternExport = 'DataBase/LOS_DB.xlsx'

In [72]:
def limits_of_stability(file):
    
    # Sample frequency, and frequency cut for filter
    fs = 100
    fc = 7
    
    # Open file
    columns = ['Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz']
    df = pd.read_csv(file, skiprows=4, header=None, names=columns, encoding="Windows-1252")
       
    #Assigning timeline for data
    time = df.index.values / fs
    df['Time'] = time
        
    #Filter definition
    b, a = signal.butter(4, fc/(fs/2), 'low', analog=False)

    #Filtering data
    df['Fx'] = signal.filtfilt(b,a, df['Fx'])
    df['Fy'] = signal.filtfilt(b,a, df['Fy'])
    df['Fz'] = signal.filtfilt(b,a, df['Fz'])
    df['Mx'] = signal.filtfilt(b,a, df['Mx'])
    df['My'] = signal.filtfilt(b,a, df['My'])
    df['Mz'] = signal.filtfilt(b,a, df['Mz'])
        
    #Define COP in both directions
    cpx = -(df['My'].values / df['Fz'].values)*100
    df['COPX'] = cpx
    cpy = -(-df['Mx'].values / df['Fz'] * 100)
    df['COPY'] = cpy
        
    #Values to start searching for crossection of linear regression
    ph1 = 800
    ph2 = 1500
    dh = 500
        
    # Get coefficients
    a1, b1 = np.polyfit(df['Time'][:ph1].values, df['COPY'][:ph1].values, 1) 
    a3, b3 = np.polyfit(df['Time'][ph2:].values, df['COPY'][ph2:], 1)

    # Get regresion lines
    ywh1 = a1 * df['Time'].values + b1
    ywh3 = a3 * df['Time'].values + b3
        
    # Adjust positions of start/end of phases
    pph2 = 0
    pph1 = 0

    # Find the point where regression line cross the moment of forward lean
    for ind in range(ph1, len(df)):
        if (ywh3[ind] - df['COPY'][ind]) > 0:
            pph2 = ind
        else:
            if (abs(ywh3[ind] - df['COPY'][ind]) < abs( ywh3[ind-1] - df['COPY'][ind-1])):
                pph2 = ind
                break    

    # Same for 1st phase
    for ind in range(ph2, 0, -1):
        if (ywh1[ind] - df['COPY'][ind]) < 0:
            pph1 = ind
        else:
            if ( abs(ywh1[ind] - df['COPY'][ind]) < abs(ywh1[ind+1] - df['COPY'][ind+1]) ):
                pph1 = ind
                break    
                    
    # Find regression line for 2nd phase
    a2, b2 = np.polyfit(df['Time'][pph1:pph2].values, df['COPY'][pph1:pph2].values, 1)
    ywh2 = b2 + a2 * df['Time'][pph1:pph2]
        
    # Define regression line again with new time limits
    a1, b1 = np.polyfit(df['Time'][:pph1].values, df['COPY'][:pph1].values, 1)
    a3, b3 = np.polyfit(df['Time'][pph2:].values, df['COPY'][pph2:].values, 1)

    # Get regresion lines
    ywh1 = a1 * df['Time'].values + b1
    ywh3 = a3 * df['Time'].values + b3
        
    #Calculating COP characteristics in anterior-posterior direction (AP) Phase 1
    f1_raCOP_AP = df['COPY'][:pph1].max() - df['COPY'][:pph1].min()
    f1_mean_AP = df['COPY'][:pph1].mean()
    f1_stdCOP_AP = df['COPY'][:pph1].std()
    f1_max_AP = df['COPY'][:pph1].max()
    f1_min_AP = df['COPY'][:pph1].min()
    f1_lenCOP_AP = 0

    for i in range(0, len(df['COPY'][:pph1])) :
        f1_lenCOP_AP += abs(df['COPY'][i+1] - (df['COPY'][i]) ) 

    f1_vCOP_AP = f1_lenCOP_AP / ( len(df['COPY'][:pph1]) / fs )     
    f1_sampeEn_AP = ent.sample_entropy(df['COPY'][:pph1], 3, 0.2)

    #Calculating COP characteristics in anterior-posterior direction (AP) Phase 2
    f2_raCOP_AP = df['COPY'][pph1:pph2].max() - df['COPY'][pph1:pph2].min()
    f2_mean_AP = df['COPY'][pph1:pph2].mean()
    f2_stdCOP_AP = df['COPY'][pph1:pph2].std()
    f2_max_AP = df['COPY'][pph1:pph2].max()
    f2_min_AP = df['COPY'][pph1:pph2].min()

    #Calculating COP characteristics in anterior-posterior direction (AP) Phase 3
    f3_raCOP_AP = df['COPY'][pph2:].max() - df['COPY'][pph2:].min()
    f3_mean_AP = df['COPY'][pph2:].mean()
    f3_stdCOP_AP = df['COPY'][pph2:].std()
    f3_max_AP = df['COPY'][pph2:].max()
    f3_min_AP = df['COPY'][pph2:].min()
    f3_lenCOP_AP = 0

    for i in range(pph2, len(df['COPY'])-1) :
        f3_lenCOP_AP += abs(df['COPY'][i+1] - (df['COPY'][i]) ) 

    f3_vCOP_AP = f3_lenCOP_AP / ( len(df['COPY'][pph2:]) / fs )     
    f3_sampeEn_AP = ent.sample_entropy(df['COPY'][pph2:], 3, 0.2)

    #Calculating COP characteristics in anterior-posterior direction (ML) Phase 1
    f1_raCOP_ML = df['COPX'][:pph1].max() - df['COPX'][:pph1].min()
    f1_mean_ML = df['COPX'][:pph1].mean()
    f1_stdCOP_ML = df['COPX'][:pph1].std()
    f1_max_ML = df['COPX'][:pph1].max()
    f1_min_ML = df['COPX'][:pph1].min()
    f1_lenCOP_ML = 0
   
    for i in range(0, len(df['COPX'][:pph1])) :
        f1_lenCOP_ML += abs(df['COPX'][i+1] - (df['COPX'][i]) ) 

    f1_vCOP_ML = f1_lenCOP_ML / ( len(df['COPX'][:pph1]) / fs )     
    f1_sampeEn_ML = ent.sample_entropy(df['COPX'][:pph1], 3, 0.2)

    #Calculating COP characteristics in anterior-posterior direction (ML) Phase 3
    f3_raCOP_ML = df['COPX'][pph2:].max() - df['COPX'][pph2:].min()
    f3_mean_ML = df['COPX'][pph2:].mean()
    f3_stdCOP_ML = df['COPX'][pph2:].std()
    f3_max_ML = df['COPX'][pph2:].max()
    f3_min_ML = df['COPX'][pph2:].min()
    f3_lenCOP_ML = 0
   
    for i in range(pph2, len(df['COPX'])-1) :
        f3_lenCOP_ML += abs(df['COPX'][i+1] - (df['COPX'][i]) ) 

    f3_vCOP_ML = f3_lenCOP_ML / ( len(df['COPX'][pph2:]) / fs )     
    f3_sampeEn_ML = ent.sample_entropy(df['COPX'][pph2:], 3, 0.2)
        
    # Define variables for LOS - time of phases and max lean forwarard value
    T1P = 0.0
    T1K = (pph1-1) / 100
    T2P = pph1 / 100
    T2K = (pph2-1) / 100
    T3P = pph2 / 100
    T3K = len(df) / 100
    R1 = df['COPY'][pph2:].mean() - df['COPY'][:pph1].mean()
    R3 = df['COPY'].max() - df['COPY'].min()
        
    # Get name and trial number from file path
    fname = file.replace(patternFname  , '')
    fnameSplit = fname.split("_")
    t_num = fnameSplit[0]
    s_num = fnameSplit[2].split(".")
    name = "_".join([fnameSplit[1], s_num[0]])
        
    # Saving figure tagged with subject name and trial number
    fig = plt.figure(figsize = (12,6))
    plt.style.use('ggplot')
    ax = fig.add_subplot(111)
    ax.plot(df['Time'], df['COPY'], linewidth = 2, color = 'dodgerblue')
    ax.plot(df['Time'][:pph1], ywh1[:pph1], color='red', linewidth=1.5)
    ax.plot(df['Time'][pph1:pph2], ywh2, color='red', linewidth=1.5)
    ax.plot(df['Time'][pph2:], ywh3[pph2:], color='red', linewidth=1.5)
    ax.set(title="Trial nr: " + str(t_num) + '; ' + ' Subject: ' + name , xlabel = 'Time[s]', ylabel = 'Center of pressure[cm]')
    fig.savefig(patternCharts + str(t_num) + '_' + name + '.png')
    plt.close()
        
    # Saving results into list of dictionaries
    res_ap = {'Subject': name, 'Trial': t_num, 'F1_raCOP': f1_raCOP_AP, 'F1_stdCOP': f1_stdCOP_AP, 'F1_lenCOP': f1_lenCOP_AP, 
          'F1_vCOP': f1_vCOP_AP, 'F1_sampEn': f1_sampeEn_AP[0], 'F1_stdSampEn': f1_sampeEn_AP[1], 'F3_raCOP': f3_raCOP_AP, 
          'F3_stdCOP': f3_stdCOP_AP, 'F3_lenCOP': f3_lenCOP_AP, 'F3_vCOP': f3_vCOP_AP, 'F3_sampEn': f3_sampeEn_AP[0], 
          'F3_stdSampEn': f3_sampeEn_AP[1]}
    res_ml = {'Subject': name, 'Trial': t_num, 'F1_raCOP': f1_raCOP_ML, 'F1_stdCOP': f1_stdCOP_ML, 'F1_lenCOP': f1_lenCOP_ML, 
          'F1_vCOP': f1_vCOP_ML, 'F1_sampEn': f1_sampeEn_ML[0], 'F1_stdSampEn': f1_sampeEn_ML[1], 'F3_raCOP': f3_raCOP_ML, 
          'F3_stdCOP': f3_stdCOP_ML, 'F3_lenCOP': f3_lenCOP_ML, 'F3_vCOP': f3_vCOP_ML, 'F3_sampEn': f3_sampeEn_ML[0], 
          'F3_stdSampEn': f3_sampeEn_ML[1]}
    res_los = {'Subject': name, 'Trial': t_num, 'T1P': T1P, 'T1K': T1K, 'B1': a1, 'Mean1': f1_mean_AP, 'Std1': f1_stdCOP_AP, 
           'Max1': f1_max_AP, 'Min1': f1_min_AP, 'Range1': f1_raCOP_AP, 'T2P': T2P, 'T2K': T2K, 'B2': a2, 'Mean2': f2_mean_AP, 
           'Std2': f2_stdCOP_AP, 'Max2': f2_max_AP, 'Min2': f2_min_AP, 'Range2': f2_raCOP_AP, 'T3P': T3P, 'T3K': T3K, 'B3': a3, 
           'Mean3': f3_mean_AP, 'Std3': f3_stdCOP_AP, 'Max3': f3_max_AP, 'Min3': f3_min_AP, 'Range3': f3_raCOP_AP, 
           'R1': R1, 'R3': R3}
        
    res_bundle = [res_ap, res_ml, res_los]
        
    return res_bundle 

In [73]:
# Definition of Result dataframes
df_ap = pd.DataFrame(columns=['Subject', 'Trial', 'F1_raCOP', 'F1_stdCOP', 'F1_lenCOP', 'F1_vCOP', 'F1_sampEn', 'F1_stdSampEn',
                                                'F3_raCOP', 'F3_stdCOP', 'F3_lenCOP', 'F3_vCOP', 'F3_sampEn', 'F3_stdSampEn'])
df_ml = pd.DataFrame(columns=['Subject', 'Trial','F1_raCOP', 'F1_stdCOP', 'F1_lenCOP', 'F1_vCOP', 'F1_sampEn', 'F1_stdSampEn',
                                               'F3_raCOP', 'F3_stdCOP', 'F3_lenCOP', 'F3_vCOP', 'F3_sampEn', 'F3_stdSampEn'])
df_los = pd.DataFrame(columns=['Subject', 'Trial','T1P', 'T1K', 'B1', 'Mean1', 'Std1', 'Max1', 'Min1', 'Range1',
                                                'T2P', 'T2K', 'B2', 'Mean2', 'Std2', 'Max2', 'Min2', 'Range2',
                                                'T3P', 'T3K', 'B3', 'Mean3', 'Std3', 'Max3', 'Min3', 'Range3', 'R1', 'R3'])

In [74]:
# Getting all results

files = glob.glob(patternFiles)

#Counter for files (only for information about progress)
fileCount = 1
fileAmount = len(files)

for file in files:
    #print(file)
    data = limits_of_stability(file)
    df_ap = pd.concat([df_ap, pd.DataFrame([data[0]])], ignore_index=True)
    df_ml = pd.concat([df_ml, pd.DataFrame([data[1]])], ignore_index=True)
    df_los = pd.concat([df_los, pd.DataFrame([data[2]])], ignore_index=True)
    
    
    #Information about progress
    print(color.BOLD + 'Total progress : ' + color.END + str(int(fileCount/fileAmount  *100)) + '%', end = '\r')
    fileCount = fileCount + 1

/tmp/ipykernel_14009/2988541185.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_ap = pd.concat([df_ap, pd.DataFrame([data[0]])], ignore_index=True)
/tmp/ipykernel_14009/2988541185.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_ml = pd.concat([df_ml, pd.DataFrame([data[1]])], ignore_index=True)
/tmp/ipykernel_14009/2988541185.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude e

In [75]:
df_ap.sort_values(by=["Subject", "Trial"], inplace=True)
df_ml.sort_values(by=["Subject", "Trial"], inplace=True)
df_los.sort_values(by=["Subject", "Trial"], inplace=True)

In [76]:
with pd.ExcelWriter(patternExport) as writer:  
    df_ap.to_excel(writer, sheet_name='AP', index=False)
    df_ml.to_excel(writer, sheet_name='ML', index=False)
    df_los.to_excel(writer, sheet_name='LOS', index=False)